# 🤖 Agentes e Ferramentas: Quando a IA Ganha Superpoderes!

**Módulo 9 - LangChain v0.2 com Pedro Guth**

Bora! Chegamos numa parte SENSACIONAL do LangChain! 🚀

Imagina que você tem um assistente pessoal que não só conversa contigo, mas também pode:
- Buscar informações na internet
- Fazer cálculos complexos
- Acessar APIs
- Executar código
- E muito mais!

Isso são os **Agents** (Agentes) com **Tools** (Ferramentas)! É como dar uma caixa de ferramentas completa para sua IA! 🧰

## 🎯 O que vamos aprender?

- O que são Agents e Tools
- Como criar ferramentas customizadas
- Diferentes tipos de agentes
- ReAct (Reasoning + Acting)
- Integração com APIs externas
- Casos práticos e projetos

**Dica do Pedro:** Os Agents são o que mais aproxima a IA de um "assistente inteligente" de verdade!

In [ ]:
# Vamos instalar e importar tudo que precisamos!
!pip install langchain langchain-google-genai langchain-community wikipedia-api requests

import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_react_agent, AgentExecutor, Tool
from langchain.prompts import PromptTemplate
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import requests
import json
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

print("✅ Imports realizados com sucesso!")

In [ ]:
# Configurando nossa API do Google (use sua chave!)
os.environ["GOOGLE_API_KEY"] = "sua_chave_aqui"  # Substitua pela sua chave!

# Inicializando nosso modelo
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.1
)

print("🤖 Modelo Gemini carregado e pronto!")

## 🧠 Conceitos Fundamentais

### O que são Agents?

Tá, mas o que é um **Agent** exatamente?

Um Agent é como um "cérebro" que pode:
1. **Raciocinar** sobre um problema
2. **Decidir** qual ferramenta usar
3. **Executar** a ação
4. **Avaliar** o resultado
5. **Repetir** até resolver o problema

### O que são Tools?

As **Tools** são as "mãos" do Agent! São funções que permitem ao Agent:
- Buscar na web
- Fazer cálculos
- Acessar bancos de dados
- Executar código
- E qualquer coisa que você programar!

**Analogia:** É como dar um smartphone para sua IA - ela pode usar diferentes apps para resolver problemas!

## 🔧 Criando Nossa Primeira Tool

Vamos começar criando uma ferramenta simples - uma calculadora!

O padrão ReAct funciona assim:
- **Re**asoning (Raciocínio): "Preciso calcular algo"
- **Act**ion (Ação): "Vou usar a calculadora"
- **Observation** (Observação): "O resultado foi X"

$$\text{Agent} = \text{Reasoning} + \text{Action} + \text{Observation}$$

In [ ]:
# Criando nossa primeira tool - Calculadora!
def calculadora(expressao):
    """
    Executa cálculos matemáticos seguros.
    Entrada: expressão matemática como string
    Saída: resultado do cálculo
    """
    try:
        # Por segurança, vamos limitar as operações
        allowed_chars = set('0123456789+-*/.() ')
        if not set(expressao).issubset(allowed_chars):
            return "Erro: Caracteres não permitidos na expressão"
        
        resultado = eval(expressao)
        return f"O resultado de {expressao} é: {resultado}"
    except Exception as e:
        return f"Erro no cálculo: {str(e)}"

# Testando nossa calculadora
print(calculadora("2 + 3 * 4"))
print(calculadora("(10 + 5) / 3"))

In [ ]:
# Criando uma tool de data e hora
def obter_data_hora(formato="completo"):
    """
    Obtém a data e hora atual.
    Entrada: formato ('completo', 'data', 'hora')
    Saída: data/hora formatada
    """
    agora = datetime.now()
    
    if formato == "data":
        return agora.strftime("%d/%m/%Y")
    elif formato == "hora":
        return agora.strftime("%H:%M:%S")
    else:
        return agora.strftime("%d/%m/%Y às %H:%M:%S")

# Testando
print("Data completa:", obter_data_hora())
print("Só a data:", obter_data_hora("data"))
print("Só a hora:", obter_data_hora("hora"))

## 🛠️ Transformando Funções em Tools do LangChain

Agora vem a mágica! Vamos transformar nossas funções em Tools que o Agent pode usar.

**Dica do Pedro:** Cada Tool precisa de um nome descritivo e uma descrição clara - o Agent usa isso para decidir quando usar cada ferramenta!

In [ ]:
# Criando as Tools do LangChain
tool_calculadora = Tool(
    name="Calculadora",
    func=calculadora,
    description="Útil para fazer cálculos matemáticos. Entrada: expressão matemática como string (ex: '2+3*4')"
)

tool_data_hora = Tool(
    name="DataHora", 
    func=obter_data_hora,
    description="Obtém data e hora atual. Entrada: 'completo', 'data' ou 'hora'"
)

# Tool do Wikipedia (já pronta do LangChain!)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang="pt"))

tool_wikipedia = Tool(
    name="Wikipedia",
    func=wikipedia.run,
    description="Busca informações na Wikipedia em português. Entrada: termo de busca"
)

print("🛠️ Tools criadas com sucesso!")
print(f"Tool 1: {tool_calculadora.name}")
print(f"Tool 2: {tool_data_hora.name}")
print(f"Tool 3: {tool_wikipedia.name}")

## 🎭 Criando Nosso Primeiro Agent

Agora vamos criar o Agent! Ele vai usar o padrão ReAct para raciocinar e agir.

O Agent precisa de:
1. **Prompt Template** - Como ele deve pensar
2. **Tools** - O que ele pode fazer  
3. **LLM** - O "cérebro" (nosso Gemini)
4. **Executor** - Quem coordena tudo

In [ ]:
# Lista de todas as nossas tools
tools = [tool_calculadora, tool_data_hora, tool_wikipedia]

# Criando um prompt customizado para nosso agent
prompt_template = """
Você é um assistente inteligente que pode usar ferramentas para ajudar o usuário.

Você tem acesso às seguintes ferramentas:
{tools}

Use o seguinte formato:

Question: a pergunta do usuário
Thought: você sempre deve pensar sobre o que fazer
Action: a ação a ser tomada, deve ser uma de [{tool_names}]
Action Input: a entrada para a ação
Observation: o resultado da ação
... (esse ciclo Thought/Action/Action Input/Observation pode se repetir N vezes)
Thought: Eu agora sei a resposta final
Final Answer: a resposta final para a pergunta original

Comece!

Question: {input}
Thought: {agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)

print("📝 Prompt do Agent criado!")

In [ ]:
# Criando o Agent ReAct
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Criando o Executor (quem vai rodar tudo)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Para ver o que está acontecendo
    max_iterations=5,  # Máximo de iterações
    handle_parsing_errors=True
)

print("🤖 Agent criado e pronto para uso!")
print(f"Tools disponíveis: {[tool.name for tool in tools]}")

## 🚀 Testando Nosso Agent

Liiindo! Agora vamos ver nosso Agent em ação! 

Vamos fazer alguns testes para ver como ele raciocina e usa as ferramentas.

**Dica do Pedro:** Repare como o Agent vai "pensar" em voz alta - isso é o ReAct funcionando!

In [ ]:
# Teste 1: Cálculo simples
pergunta1 = "Quanto é 15 * 23 + 100?"

print("🔥 TESTE 1: Cálculo matemático")
print(f"Pergunta: {pergunta1}")
print("=" * 50)

resposta1 = agent_executor.invoke({"input": pergunta1})
print(f"\n✅ Resposta final: {resposta1['output']}")

In [ ]:
# Teste 2: Informação + Data
pergunta2 = "Me fale sobre Python (linguagem de programação) e me diga que horas são agora"

print("🔥 TESTE 2: Múltiplas ferramentas")
print(f"Pergunta: {pergunta2}")
print("=" * 50)

resposta2 = agent_executor.invoke({"input": pergunta2})
print(f"\n✅ Resposta final: {resposta2['output']}")

## 📊 Visualizando o Fluxo do Agent

In [ ]:
# Vamos criar um gráfico para mostrar como funciona o fluxo do Agent
fig, ax = plt.subplots(figsize=(12, 8))

# Dados para o gráfico de fluxo
steps = ['Input', 'Reasoning', 'Tool Selection', 'Action', 'Observation', 'Final Answer']
positions = np.arange(len(steps))
complexities = [1, 3, 2, 4, 3, 1]  # Complexidade de cada etapa

# Criando o gráfico
bars = ax.bar(positions, complexities, 
              color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD'])

# Customizações
ax.set_xlabel('Etapas do Agent ReAct', fontsize=12)
ax.set_ylabel('Complexidade', fontsize=12)
ax.set_title('🤖 Fluxo de Processamento do Agent', fontsize=14, fontweight='bold')
ax.set_xticks(positions)
ax.set_xticklabels(steps, rotation=45, ha='right')

# Adicionando valores nas barras
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.05,
            f'{complexities[i]}', ha='center', va='bottom')

plt.tight_layout()
plt.grid(True, alpha=0.3)
plt.show()

print("📊 Este gráfico mostra a complexidade de cada etapa do processamento!")

## 🌐 Criando uma Tool de API Externa

Agora vamos criar algo mais avançado - uma tool que consulta uma API externa!

Vamos usar a API do ViaCEP para buscar informações de CEP.

In [ ]:
# Tool para consultar CEP
def consultar_cep(cep):
    """
    Consulta informações de um CEP brasileiro.
    Entrada: CEP no formato 12345678 ou 12345-678
    Saída: Informações do endereço
    """
    try:
        # Limpando o CEP
        cep_limpo = cep.replace("-", "").replace(".", "")
        
        if len(cep_limpo) != 8 or not cep_limpo.isdigit():
            return "Erro: CEP deve ter 8 dígitos"
        
        # Fazendo a requisição
        url = f"https://viacep.com.br/ws/{cep_limpo}/json/"
        response = requests.get(url, timeout=5)
        
        if response.status_code == 200:
            dados = response.json()
            
            if "erro" in dados:
                return f"CEP {cep} não encontrado"
            
            return (f"CEP {dados['cep']}: {dados['logradouro']}, "
                   f"{dados['bairro']}, {dados['localidade']}-{dados['uf']}")
        else:
            return "Erro ao consultar o CEP"
            
    except Exception as e:
        return f"Erro na consulta: {str(e)}"

# Testando a função
print("🏠 Testando consulta de CEP:")
print(consultar_cep("01310-100"))  # Av. Paulista, SP

In [ ]:
# Criando a tool de CEP e atualizando nosso agent
tool_cep = Tool(
    name="ConsultarCEP",
    func=consultar_cep,
    description="Consulta informações de endereço por CEP brasileiro. Entrada: CEP (ex: '12345-678' ou '12345678')"
)

# Nova lista de tools (com CEP incluído)
tools_completas = [tool_calculadora, tool_data_hora, tool_wikipedia, tool_cep]

# Recriando o agent com a nova tool
agent_completo = create_react_agent(
    llm=llm,
    tools=tools_completas,
    prompt=prompt
)

agent_executor_completo = AgentExecutor(
    agent=agent_completo,
    tools=tools_completas,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True
)

print("🚀 Agent atualizado com tool de CEP!")
print(f"Tools disponíveis: {[tool.name for tool in tools_completas]}")

## 🎯 Teste Completo do Agent

Agora vamos fazer um teste mais complexo que usa múltiplas ferramentas!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-09_img_01.png)

In [ ]:
# Teste complexo: múltiplas ferramentas
pergunta_complexa = """
Preciso de ajuda com o seguinte:
1. Calcule quantos dias se passaram desde 1º de janeiro de 2024 até hoje
2. Me diga informações sobre o CEP 04038-001
3. Busque informações sobre inteligência artificial
"""

print("🔥 TESTE COMPLEXO: Múltiplas ferramentas")
print(f"Pergunta: {pergunta_complexa}")
print("=" * 60)

resposta_complexa = agent_executor_completo.invoke({"input": pergunta_complexa})
print(f"\n✅ Resposta final: {resposta_complexa['output']}")

## 📊 Gráfico de Performance das Tools

In [ ]:
# Simulando dados de uso das tools
tools_names = ['Calculadora', 'Data/Hora', 'Wikipedia', 'CEP']
uso_simulado = [25, 15, 35, 20]  # Percentual de uso
velocidade = [0.1, 0.05, 2.5, 1.2]  # Tempo médio em segundos

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Uso das Tools (Pizza)
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
ax1.pie(uso_simulado, labels=tools_names, autopct='%1.1f%%', 
        colors=colors, startangle=90)
ax1.set_title('📊 Distribuição de Uso das Tools', fontsize=12, fontweight='bold')

# Gráfico 2: Velocidade das Tools (Barras)
bars = ax2.bar(tools_names, velocidade, color=colors)
ax2.set_title('⚡ Tempo Médio de Resposta (segundos)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Tempo (s)')
ax2.tick_params(axis='x', rotation=45)

# Adicionando valores nas barras
for bar, vel in zip(bars, velocidade):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{vel}s', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("📈 Análise de performance das nossas tools!")

## 🎨 Diagrama da Arquitetura

```mermaid
graph TD
    A[Usuário] --> B[Agent Executor]
    B --> C[LLM - Gemini]
    C --> D{Precisa de Tool?}
    D -->|Sim| E[Selecionar Tool]
    D -->|Não| F[Resposta Final]
    E --> G[Calculadora]
    E --> H[Data/Hora]
    E --> I[Wikipedia]
    E --> J[CEP]
    G --> K[Resultado]
    H --> K
    I --> K
    J --> K
    K --> C
    F --> L[Retorno ao Usuário]
```

## 🎓 Exercício Prático 1

**Desafio:** Crie uma tool personalizada!

Sua missão:
1. Crie uma função que converte temperaturas (Celsius ↔ Fahrenheit)
2. Transforme em uma Tool do LangChain
3. Adicione ao nosso agent
4. Teste com uma pergunta

**Dica do Pedro:** A fórmula é: °F = (°C × 9/5) + 32 e °C = (°F - 32) × 5/9

In [ ]:
# SEU CÓDIGO AQUI!
# Implemente a tool de conversão de temperatura

def converter_temperatura(entrada):
    """
    Converte temperaturas entre Celsius e Fahrenheit.
    Entrada: "25C" para converter de Celsius ou "77F" para Fahrenheit
    Saída: temperatura convertida
    """
    # COMPLETE AQUI!
    pass

# Teste sua função aqui
# print(converter_temperatura("25C"))
# print(converter_temperatura("77F"))

print("💪 Complete o exercício e teste sua tool!")

## 🏆 Exercício Prático 2 - Desafio Avançado

**Super Desafio:** Crie um agent especializado!

Crie um agent que seja um "Assistente de Viagem" com as seguintes tools:
1. Consultar CEP (já temos)
2. Buscar informações sobre cidades (Wikipedia)
3. Calcular distâncias (você cria)
4. Converter moedas (você cria - use uma API gratuita)

**Exemplo de uso:**
"Quero viajar para uma cidade com CEP 20040-020. Me fale sobre essa cidade e converta R$ 1000 para dólares."

In [ ]:
# SEU CÓDIGO AQUI!
# Crie o Assistente de Viagem

def converter_moeda(valor_moeda):
    """
    Converte valores entre moedas (simplificado).
    Entrada: "100 BRL USD" (100 reais para dólares)
    """
    # IMPLEMENTE AQUI!
    # Dica: Use uma API como exchangerate-api.com (gratuita)
    pass

def calcular_distancia(origem_destino):
    """
    Calcula distância entre cidades (simplificado).
    Entrada: "São Paulo Rio de Janeiro"
    """
    # IMPLEMENTE AQUI!
    # Pode ser uma versão simplificada com distâncias fixas
    pass

print("🌍 Crie seu Assistente de Viagem aqui!")

## 🔍 Monitorando e Debugando Agents

Uma parte importante é saber o que está acontecendo "por baixo do capô".

**Dica do Pedro:** Sempre use `verbose=True` durante desenvolvimento!

In [ ]:
# Função para analisar o comportamento do agent
def analisar_agent_steps(pergunta):
    """
    Executa uma pergunta e mostra estatísticas.
    """
    print(f"🔍 ANÁLISE DA PERGUNTA: {pergunta}")
    print("=" * 60)
    
    # Executando e capturando
    resultado = agent_executor_completo.invoke({"input": pergunta})
    
    # Simulando estatísticas (em uma implementação real, você capturaria isso)
    stats = {
        "tools_usadas": 2,
        "iteracoes": 3,
        "tempo_total": "2.5s",
        "tokens_usados": 450
    }
    
    print(f"\n📊 ESTATÍSTICAS:")
    for key, value in stats.items():
        print(f"  {key}: {value}")
    
    return resultado

# Testando nossa análise
resultado_analise = analisar_agent_steps("Que horas são e quanto é 100 / 4?")

## 🚀 Performance e Otimizações

Algumas dicas para deixar seus Agents mais rápidos e eficientes:

### ⚡ Dicas de Performance:
1. **Limite iterações:** `max_iterations=3` (evita loops infinitos)
2. **Cache de resultados:** Para APIs externas
3. **Timeout nas tools:** Para evitar travamentos
4. **Descrições claras:** Ajuda o LLM a escolher melhor
5. **Temperature baixa:** Para decisões mais determinísticas

### 🛡️ Dicas de Segurança:
1. **Validação de entrada:** Sempre validar dados de entrada
2. **Rate limiting:** Para APIs externas
3. **Sanitização:** Para evitar injeções
4. **Logs detalhados:** Para auditoria

In [ ]:
# Exemplo de tool com validação e cache
import time
from functools import lru_cache

@lru_cache(maxsize=100)  # Cache dos últimos 100 resultados
def buscar_info_cache(termo):
    """
    Busca informações com cache para otimizar performance.
    """
    # Validação
    if not termo or len(termo.strip()) < 2:
        return "Erro: Termo de busca muito curto"
    
    # Simulando busca (na prática seria uma API real)
    time.sleep(0.5)  # Simula latência
    return f"Informações sobre '{termo}' (resultado em cache)"

# Testando cache
print("⏱️ Primeira busca (sem cache):")
start = time.time()
result1 = buscar_info_cache("Python")
print(f"Tempo: {time.time() - start:.2f}s - {result1}")

print("\n⚡ Segunda busca (com cache):")
start = time.time()
result2 = buscar_info_cache("Python")
print(f"Tempo: {time.time() - start:.2f}s - {result2}")

## 📈 Comparação: Com vs Sem Agents

Vamos ver a diferença entre usar um chatbot simples e um Agent com tools!

In [ ]:
# Comparando abordagens
fig, ax = plt.subplots(figsize=(12, 7))

# Dados da comparação
categorias = ['Precisão\nMatemática', 'Info\nAtualizada', 'Versatilidade', 'Confiabilidade', 'Velocidade']
chatbot_simples = [3, 2, 4, 7, 9]
agent_tools = [10, 9, 9, 8, 7]

x = np.arange(len(categorias))
width = 0.35

# Criando as barras
bars1 = ax.bar(x - width/2, chatbot_simples, width, label='Chatbot Simples', 
               color='#FF6B6B', alpha=0.8)
bars2 = ax.bar(x + width/2, agent_tools, width, label='Agent + Tools', 
               color='#4ECDC4', alpha=0.8)

# Customizações
ax.set_xlabel('Aspectos Avaliados')
ax.set_ylabel('Score (1-10)')
ax.set_title('🥊 Chatbot Simples vs Agent com Tools', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(categorias)
ax.legend()
ax.set_ylim(0, 11)

# Adicionando valores
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{int(height)}', ha='center', va='bottom')

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 Os Agents claramente ganham na maioria dos aspectos!")

## 🎯 Casos de Uso Reais

### 💼 Onde usar Agents na prática:

1. **Assistentes Empresariais:**
   - Consultar ERP/CRM
   - Gerar relatórios
   - Automatizar tarefas

2. **Atendimento ao Cliente:**
   - Consultar pedidos
   - Verificar estoque
   - Processar devoluções

3. **Análise de Dados:**
   - Executar consultas SQL
   - Gerar gráficos
   - Criar dashboards

4. **Automação Residencial:**
   - Controlar IoT
   - Monitorar sistemas
   - Otimizar consumo

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-09_img_02.png)

## 🔮 Preparando para os Próximos Módulos

Nos próximos módulos vamos usar tudo que aprendemos aqui:

### 🚀 Módulo 10 e 11 - Projetos Finais:
- Vamos criar projetos completos usando Agents
- Integração com RAG (módulo 8) + Agents
- Memory Systems (módulo 5) + Agents

### 📦 Módulo 12 - Deploy:
- Como colocar Agents em produção
- Streamlit + Agents
- Monitoramento e logs

### 🆕 Módulo 14 - LangGraph:
- Evolução dos Agents
- Workflows mais complexos
- Estados e transições

**Dica do Pedro:** Os Agents são a base para construir assistentes realmente inteligentes!

## 🎉 Resumo do Módulo

### 🧠 O que aprendemos:

1. **Conceitos Fundamentais:**
   - Agents vs Chatbots simples
   - Padrão ReAct (Reasoning + Acting)
   - Tools e sua importância

2. **Implementação Prática:**
   - Criar tools customizadas
   - Configurar agents ReAct
   - Integrar APIs externas
   - Monitorar e debugar

3. **Tools que Criamos:**
   - ✅ Calculadora matemática
   - ✅ Data e hora atual
   - ✅ Consulta Wikipedia
   - ✅ Consulta de CEP

### 🚀 Próximos Passos:
- Experimente criar suas próprias tools
- Integre com APIs que você usa
- Prepare-se para os projetos finais!

**Liiindo! Agora você sabe criar Agents inteligentes! 🎊**

Nos vemos no próximo módulo para começar nosso projeto final! Bora colocar a mão na massa! 💪